## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [1]:

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17086051013130506476
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4952306483
locality {
  bus_id: 1
  links {
  }
}
incarnation: 690894396331312888
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


In [3]:
X_train = np.load("X_train.dat")
y_train = np.load("y_train.dat")

In [7]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=2)
X_train = X_train.reshape(X_train.shape[0], 174, 124, 1)

In [8]:
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

training_set_size = 8000

# split data into validation set
X_train_validation = X_train[training_set_size:, :,:,:]
y_train_validation = y_train[training_set_size:, :]
y_train = y_train[:training_set_size,:]
X_train = X_train[:training_set_size, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(9000, 174, 124, 1)
(9000, 10)
Validation x train set:(1000, 174, 124, 1)
X train set:(8000, 174, 124, 1)
Validation y train set:(1000, 10)
Y train set:(8000, 10)


## Build the CNN network

In [10]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (7,7), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, (7,7), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (7, 7), strides=(1,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

## Begin training the model

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("best_model_100epochs", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)

model.compile(adam, 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, epochs=100 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 8000 samples, validate on 1000 samples
Epoch 1/100
8000/8000 [==============================] - 38s 5ms/step - loss: 2.0580 - acc: 0.3103 - val_loss: 5.1223 - val_acc: 0.2620
Epoch 2/100
8000/8000 [==============================] - 32s 4ms/step - loss: 1.7434 - acc: 0.3621 - val_loss: 3.0879 - val_acc: 0.2640
Epoch 3/100
7232/8000 [==========================>...] - ETA: 2s - loss: 1.5761 - acc: 0.4269

In [58]:
model.save("trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]


In [51]:
from keras import backend as K
K.clear_session()